In [1]:
# Import dependencies
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, load_img

print(tf.__version__)

2.12.0


In [2]:
# Connect to the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# extract the data from drive to the collab local
from google.colab.output import eval_js
eval_js('google.colab.output.setIframeHeight("100")')

!unzip "/content/drive/MyDrive/EE468 Project/data_splits.zip" -d "/content/"

Görüntülenen çıkış son 5000 satıra kısaltıldı.
  inflating: /content/data_splits/train/Tomato___Tomato_Yellow_Leaf_Curl_Virus/image (1333).JPG  
  inflating: /content/data_splits/train/Tomato___Tomato_Yellow_Leaf_Curl_Virus/image (5221).JPG  
  inflating: /content/data_splits/train/Tomato___Tomato_Yellow_Leaf_Curl_Virus/image (1534).JPG  
  inflating: /content/data_splits/train/Tomato___Tomato_Yellow_Leaf_Curl_Virus/image (3682).JPG  
  inflating: /content/data_splits/train/Tomato___Tomato_Yellow_Leaf_Curl_Virus/image (2466).JPG  
  inflating: /content/data_splits/train/Tomato___Tomato_Yellow_Leaf_Curl_Virus/image (1730).JPG  
  inflating: /content/data_splits/train/Tomato___Tomato_Yellow_Leaf_Curl_Virus/image (4602).JPG  
  inflating: /content/data_splits/train/Tomato___Tomato_Yellow_Leaf_Curl_Virus/image (2347).JPG  
  inflating: /content/data_splits/train/Tomato___Tomato_Yellow_Leaf_Curl_Virus/image (2148).JPG  
  inflating: /content/data_splits/train/Tomato___Tomato_Yellow_Leaf_Cur

# Start Training From Scratch

In [ ]:
# Download the inception v3 weights
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2023-05-20 14:06:31--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.199.128, 142.250.107.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.199.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   139MB/s    in 0.6s    

2023-05-20 14:06:32 (139 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [ ]:
# Import the inception model  
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Set the path for pre-trained weights
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [ ]:
# Define the train, validation, and test base directories
train_dir = '/content/data_splits/train/'
val_dir = '/content/data_splits/validation/'
test_dir = '/content/data_splits/test/'

In [ ]:
# Instantiate the ImageDataGenerator for train, validation, and test
train_datagen = ImageDataGenerator(rescale=1.0/255.0,
                                     rotation_range=50,
                                     width_shift_range=0.25,
                                     height_shift_range=0.25,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode='nearest')
  
val_datagen = ImageDataGenerator(rescale=1.0/255.0,)

test_datagen = ImageDataGenerator(rescale=1.0/255.0,)

In [ ]:
train_gen = train_datagen.flow_from_directory(directory=train_dir,
                                              batch_size=32,
                                              class_mode='categorical',
                                              target_size=(256, 256))

val_gen = val_datagen.flow_from_directory(directory=val_dir, 
                                                 batch_size=32, 
                                                 class_mode='categorical',
                                                 target_size=(256, 256))

test_gen = test_datagen.flow_from_directory(directory=test_dir,
                                            batch_size=32,
                                            class_mode='categorical',
                                            target_size=(256, 256))

Found 31231 images belonging to 39 classes.
Found 10407 images belonging to 39 classes.
Found 10407 images belonging to 39 classes.


In [ ]:
def create_pre_trained_model(local_weights_file):
  """
  Initializes an InceptionV3 model.
  
  Args:
    local_weights_file (string): path pointing to a pretrained weights H5 file
    
  Returns:
    pre_trained_model: the initialized InceptionV3 model
  """

  pre_trained_model = InceptionV3(input_shape = (256, 256, 3),
                                  include_top = False, 
                                  weights = None) 

  pre_trained_model.load_weights(local_weights_file)

  # Make all the layers in the pre-trained model non-trainable
  for layer in pre_trained_model.layers:
    layer.trainable = False

  return pre_trained_model

In [ ]:
pre_trained_model = create_pre_trained_model(local_weights_file)

# Print the model summary
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 127, 127, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 127, 127, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                      

In [ ]:
def output_of_last_layer(pre_trained_model):
  """
  Gets the last layer output of a model
  
  Args:
    pre_trained_model (tf.keras Model): model to get the last layer output from
    
  Returns:
    last_output: output of the model's last layer 
  """

  last_desired_layer = pre_trained_model.get_layer('mixed7')
  print('last layer output shape: ', last_desired_layer.output_shape)
  last_output = last_desired_layer.output
  print('last layer output: ', last_output)

  return last_output

In [ ]:
last_output = output_of_last_layer(pre_trained_model)

last layer output shape:  (None, 14, 14, 768)
last layer output:  KerasTensor(type_spec=TensorSpec(shape=(None, 14, 14, 768), dtype=tf.float32, name=None), name='mixed7/concat:0', description="created by layer 'mixed7'")


In [ ]:
def create_final_model(pre_trained_model, last_output):
  """
  Appends a custom model to a pre-trained model
  
  Args:
    pre_trained_model (tf.keras Model): model that will accept the train/test inputs
    last_output (tensor): last layer output of the pre-trained model
    
  Returns:
    model: the combined model
  """
  # Flatten the output layer to 1 dimension
  x = layers.Flatten()(last_output)
  # Add a fully connected layer with 256 hidden units and ReLU activation
  x = layers.Dense(256, activation='relu')(x) 
  # Add a final sigmoid layer for classification
  x = layers.Dense(39, activation='sigmoid')(x)        

  # Create the complete model by using the Model class
  model = Model(inputs=pre_trained_model.input, outputs=x)

  # Compile the model
  model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001), 
                loss = 'categorical_crossentropy',
                metrics = ['accuracy'])
  
  return model

In [ ]:
# Save the model in a variable
model = create_final_model(pre_trained_model, last_output)

# Inspect parameters
total_params = model.count_params()
num_trainable_params = sum([w.shape.num_elements() for w in model.trainable_weights])

print(f"There are {total_params:,} total parameters in this model.")
print(f"There are {num_trainable_params:,} trainable parameters in this model.")

There are 47,520,711 total parameters in this model.
There are 38,545,447 trainable parameters in this model.


In [ ]:
# Define a Callback class that stops training once accuracy reaches 99.95%
class AccuracyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.9995):
            print("\nReached 99.95% accuracy so cancelling training!")
            self.model.stop_training = True

In [ ]:
cp_path ='/content/drive/My Drive/EE468 Project/model/hakan_test/checkpoints/epoch{epoch:02d}.h5'

# Define callbacks
acc_callback = AccuracyCallback()
# Save model every 3 epoch in hdf5 format
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=cp_path,
                                                 period=3,
                                                 monitor="accuracy",
                                                 )

In [ ]:
history = model.fit(train_gen,
                    validation_data = val_gen,
                    epochs = 21,
                    verbose = 1,
                    callbacks=[acc_callback, cp_callback],
                    )

Epoch 1/21
976/976 [==============================] - 557s 556ms/step - loss: 0.7255 - accuracy: 0.8060 - val_loss: 0.4202 - val_accuracy: 0.8729
Epoch 2/21
976/976 [==============================] - 535s 548ms/step - loss: 0.3045 - accuracy: 0.9058 - val_loss: 0.3031 - val_accuracy: 0.9059
Epoch 3/21
976/976 [==============================] - 545s 558ms/step - loss: 0.2478 - accuracy: 0.9207 - val_loss: 0.4094 - val_accuracy: 0.8712
Epoch 4/21
976/976 [==============================] - 537s 550ms/step - loss: 0.2151 - accuracy: 0.9298 - val_loss: 0.2115 - val_accuracy: 0.9301
Epoch 5/21
976/976 [==============================] - 536s 549ms/step - loss: 0.2016 - accuracy: 0.9342 - val_loss: 0.2520 - val_accuracy: 0.9239
Epoch 6/21
976/976 [==============================] - 540s 553ms/step - loss: 0.1824 - accuracy: 0.9397 - val_loss: 0.2545 - val_accuracy: 0.9243
Epoch 7/21
976/976 [==============================] - 540s 553ms/step - loss: 0.1673 - accuracy: 0.9457 - val_loss: 0.2202 -

In [ ]:
model.save('/content/drive/My Drive/EE468 Project/model/hakan_test/trained_models/inception_v3_mixed7.h5')

# Resume Training From Checkpoint

In [ ]:
# Define the train, validation, and test base directories
train_dir = '/content/data_splits/train/'
val_dir = '/content/data_splits/validation/'
test_dir = '/content/data_splits/test/'

In [ ]:
# Instantiate the ImageDataGenerator for train, validation, and test
train_datagen = ImageDataGenerator(rescale=1.0/255.0,
                                     rotation_range=50,
                                     width_shift_range=0.25,
                                     height_shift_range=0.25,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode='nearest')
  
val_datagen = ImageDataGenerator(rescale=1.0/255.0,)

test_datagen = ImageDataGenerator(rescale=1.0/255.0,)

In [ ]:
train_gen = train_datagen.flow_from_directory(directory=train_dir,
                                              batch_size=32,
                                              class_mode='categorical',
                                              target_size=(256, 256))

val_gen = val_datagen.flow_from_directory(directory=val_dir, 
                                                 batch_size=32, 
                                                 class_mode='categorical',
                                                 target_size=(256, 256))

test_gen = test_datagen.flow_from_directory(directory=test_dir,
                                            batch_size=32,
                                            class_mode='categorical',
                                            target_size=(256, 256))

Found 31231 images belonging to 39 classes.
Found 10407 images belonging to 39 classes.
Found 10407 images belonging to 39 classes.


In [ ]:
# Model path
resumed_model_path = '/content/drive/My Drive/EE468 Project/model/hakan_test/checkpoints/epoch110.h5'

# Load the model
resumed_model = tf.keras.models.load_model(resumed_model_path)

# Print the model summary
resumed_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 127, 127, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 127, 127, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
# Print the model optimizer
resumed_model.optimizer

In [ ]:
# Define a Callback class that stops training once accuracy reaches 99.95%
class AccuracyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.9995):
            print("\nReached 99.95% accuracy so cancelling training!")
            self.model.stop_training = True

In [ ]:
cp_path ='/content/drive/My Drive/EE468 Project/model/hakan_test/checkpoints/epoch{epoch:02d}.h5'

# Define callbacks
acc_callback = AccuracyCallback()
# Save model every 3 epoch in hdf5 format
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=cp_path,
                                                 period=3,
                                                 monitor="accuracy",
                                                 )

In [ ]:
# Recompile the model just to be sure that the optimizer state has been restored properly.
resumed_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
history = resumed_model.fit(train_gen,
                    validation_data = val_gen,
                    epochs = 120,
                    verbose = 1,
                    callbacks=[acc_callback, cp_callback],
                    initial_epoch=105
                    )

Epoch 106/120
976/976 [==============================] - 558s 568ms/step - loss: 0.0280 - accuracy: 0.9905 - val_loss: 0.0571 - val_accuracy: 0.9830
Epoch 107/120
976/976 [==============================] - 565s 578ms/step - loss: 0.0229 - accuracy: 0.9920 - val_loss: 0.0568 - val_accuracy: 0.9829
Epoch 108/120
976/976 [==============================] - 565s 579ms/step - loss: 0.0259 - accuracy: 0.9914 - val_loss: 0.0563 - val_accuracy: 0.9832
Epoch 109/120
976/976 [==============================] - 562s 576ms/step - loss: 0.0235 - accuracy: 0.9915 - val_loss: 0.0559 - val_accuracy: 0.9834
Epoch 110/120
976/976 [==============================] - 565s 579ms/step - loss: 0.0254 - accuracy: 0.9913 - val_loss: 0.0558 - val_accuracy: 0.9833
Epoch 111/120
319/976 [========>.....................] - ETA: 5:50 - loss: 0.0221 - accuracy: 0.9925

In [ ]:
# Define the Model path
save_path = '/content/drive/My Drive/EE468 Project/model/hakan_test/trained_models/inception_v3_mixed7.h5'

# Save the Model
resumed_model.save(save_path)